# Welcome to glupyter-lite!

This notebook contains an example of glue-jupyter running inside of [JupyterLite](https://jupyterlite.readthedocs.io/en/stable/), which means that this is running entirely in the browser! This did require a few specific customizations that I'll describe below.

In [ ]:
import piplite

Our dependencies `fast-histogram` and `mpl-scatter-density` both require C extensions, so we need to build Pyodide-compatible wheels using Emscripten. There are a couple of other packages that didn't have compatible wheels available (note that Vispy still doesn't work yet, despite us building our own wheel, due to some runtime issues with emscripten not being recognized as an environment).

In [ ]:
await piplite.install('shapely')
await piplite.install('pypi/fast_histogram-0.14-cp39-abi3-pyodide_2024_0_wasm32.whl')
await piplite.install('pypi/mpl_scatter_density-0.8-py3-none-any.whl')
await piplite.install('pypi/glfw-2.9.0-py3-none-any.whl')
await piplite.install('pypi/vispy-0.15.2-cp312-cp312-pyodide_2024_0_wasm32.whl')

In [ ]:
# Do things like this to get around the fact that pyzmq is missing
await piplite.install("glue-core")
await piplite.install("bqplot")
await piplite.install("bqplot_image_gl")
await piplite.install("ipykernel")
await piplite.install("ipyvue")
await piplite.install("ipyvuetify")
await piplite.install("ipywidgets")


In [ ]:
await piplite.install("pypi/glue_jupyter-0.10.2.dev520+gafac49aed-py3-none-any.whl", deps=False)

In [ ]:
from src.app import App

App()